# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [6]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5].values

array([0, 0, 0, 1, 0])

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [14]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_2 = 3

# Define the number of neurons in the output layer
output_neuron = 1

In [15]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hidden_nodes_1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
nn_model.add(Dense(units=hidden_nodes_2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(Dense(units=output_neuron, activation="sigmoid"))

In [16]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [17]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 2ms/step - loss: 0.7508 - accuracy: 0.4829
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7228 - accuracy: 0.5171
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7072 - accuracy: 0.5421
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6978 - accuracy: 0.5563
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6906 - accuracy: 0.5746
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6850 - accuracy: 0.5847
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6802 - accuracy: 0.5872
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5947
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6699 - accuracy: 0.5963
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6637 - accuracy: 0.6180
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [19]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5157 - accuracy: 0.7450 - 167ms/epoch - 13ms/step
Loss: 0.5157437920570374, Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [24]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [25]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_model_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [26]:
# Make predictions with the test data
predictions = nn_model_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions[:5]

13/13 - 0s - 94ms/epoch - 7ms/step


array([[0.46053377],
       [0.31490955],
       [0.6804738 ],
       [0.65113086],
       [0.9650237 ]], dtype=float32)

In [27]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = [round(predictions[0], 0) for predictions in predictions]
predictions_df = pd.DataFrame(predictions_rounded, columns=["predictions"])
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [28]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_rounded))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       188
           1       0.77      0.75      0.76       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Here are six key data points to collect:

1. **Age** of Student - Age can influence loan options, repayment ability, terms requested, and likelihood of completing the degree.
2. **GPA** - Grade point average affects the probability of degree completion and employability, impacting available loan options.
Field of Study - The field of study affects employment prospects and default risk, influencing loan terms.
3. **Field of Study** - The field of study affects employment prospects and default risk, influencing loan terms.
4. **Degree/Certificate Applying For** - The specific degree or certificate impacts repayment plans, eligibility for certain loans, and desired terms.
College/University - The institution affects the total amount needed, as public and private universities have different financing requirements.
Employment Status of Student - Employment status (unemployed, part-time, or full-time) influences repayment schedules and loan amounts.
5. **College/University** - The institution affects the total amount needed, as public and private universities have different financing requirements.
6. **Employment Status of Student** - Employment status (unemployed, part-time, or full-time) influences repayment schedules and loan amounts.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Collaborative Filtering - This method suggests loan options to students based on similarities with other users. The data would compare students to others with similar profiles and show the loan options they selected. For example, traditional students aged 17-23 pursuing a bachelor's degree would be shown loan options that others in the same demographic have chosen.

Context-Based Filtering - This method offers recommendations based on the specific context of the student. For instance, a student who is already employed and pursuing a graduate degree or certificate would be presented with select loan options tailored to their situation.

The collected data is versatile enough to support either filtering process. Additionally, hybrid models, which combine both collaborative and context-based filtering, can often provide more comprehensive and effective recommendations.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1. **Race Discrimination:**
The recommendation system must exclude race/ethnicity data to avoid introducing bias. It's crucial not to collect this data to prevent discriminatory practices.

2. **Exclusion of Legally Protected Information:**
The system should avoid collecting any data on legally protected characteristics, including religious preferences and disabilities. Compliance with federal and state laws in both education and finance sectors is essential. Understanding and adhering to legal requirements is critical to ensure that no inappropriate data is collected or used in the recommendation process.

The system should prioritize compliance and fairness by avoiding the collection of sensitive data and adhering to all relevant legal standards.